In [1244]:
import pandas as pd
import numpy as np
from collections import namedtuple
from datetime import date
from datetime import time
from datetime import timedelta
import datetime
import pytz
import os

In [1245]:

# Load news source file and preprocess data
# Symbol_switch = correct numeric symbols with 1st non-numeric secondary symbol

def load_news(filename):
    df = pd.read_csv(filename, na_filter = False)

    # Null symbol row drop
    df = df[df['symbol'].str.len() > 0]
    #df = df[df['symbols'].str.len() > 2]

    # Number symbol replace with 1st correct secondary symbol
    df['symbols'] = df.apply(lambda row: eval(row['symbols']), axis=1)
    def symbol_switch(row):
        if row['symbol'][0].isnumeric():
            for item in row['symbols']:
                if item[0].isalpha():
                    return item
        return row['symbol']
    df['symbol'] = df.apply(symbol_switch, axis=1)
    df = df[df['symbol'].str.match('[A-Z].*')]

    # Reset index
    df.reset_index(drop = True, inplace = True)
    # Drop cols
    df2 = df.drop(['headline','hyperlink','provider_name','source_instructions','story_codes','synopsis','wire_description'],axis=1)
    
    # Update timezone
    df2['timestamp'] = [datetime.datetime.utcfromtimestamp(int(x)/(1000000)) for x in df2['timestamp']]
    df2['timestamp'] = df2['timestamp'].dt.tz_localize('UTC')
    return df2


In [1246]:
def process_news_day(df):
    listid = []
    timestamp = []
    date = []
    time = []
    guid = []
    owner = []
    action = []
    symbol = []
    description = []

    # Insert remove records if midnight > timestamp
    def clear_expired(currentdt):
        # End of day check
        for slist in symbol_lists:
            for s in list(slist.content.keys()):
                # if midnight > timestamp
                if currentdt > slist.content[s].to_pydatetime().astimezone(pytz.timezone('UTC')):
                    # Insert R action in dfout at expire_time
                    listid.append('L')
                    expire_time = slist.content[s]
                    timestamp.append(expire_time)
                    date.append(expire_time.date) # df2['date'] = df2['timestamp'].dt.date
                    time.append(expire_time.time) # df2['time'] = df2['timestamp'].dt.time
                    guid.append(slist.guid)
                    owner.append('Public')
                    action.append('R')
                    symbol.append(s)
                    description.append('Decay')
                    # Remove symbols from slist
                    del slist.content[s]  
                    
    def process_row(curr_symbol, row, slist):
        if curr_symbol not in slist.content:
            # Insert 'A' record
            listid.append('L')
            timestamp.append(row['timestamp'])
            date.append(row['timestamp'].date) # df2['date'] = df2['timestamp'].dt.date
            time.append(row['timestamp'].time) # df2['time'] = df2['timestamp'].dt.time
            guid.append(slist.guid)
            owner.append('Public')
            action.append('A')
            symbol.append(curr_symbol)
            description.append('Public')
             # Add to symbol list with expire_time = row.time + expire_duration
            slist.content[curr_symbol] = row['timestamp'] + slist.expire_duration
        else:
            # If row.time greater than expire_time
            if row['timestamp'] > slist.content[curr_symbol]:
                # Insert R action in dfout at expire_time
                listid.append('L')
                expire_time = slist.content[curr_symbol]
                timestamp.append(expire_time)
                date.append(expire_time.date) # df2['date'] = df2['timestamp'].dt.date
                time.append(expire_time.time) # df2['time'] = df2['timestamp'].dt.time
                guid.append(slist.guid)
                owner.append('Public')
                action.append('R')
                symbol.append(curr_symbol)
                description.append('Decay')
                
                # Insert A action in dfout at row.time
                listid.append('L')
                timestamp.append(row['timestamp'])
                date.append(row['timestamp'].date) # df2['date'] = df2['timestamp'].dt.date
                time.append(row['timestamp'].time) # df2['time'] = df2['timestamp'].dt.time
                guid.append(slist.guid)
                owner.append('Public')
                action.append('A')
                symbol.append(curr_symbol)
                description.append('Public')
                
            # Reset expire time for slist.content
            slist.content[curr_symbol] = row['timestamp'] + slist.expire_duration 
            
    # Set up midnight check
    # Convert est then back to utc for processing
    dt_utc = df['timestamp'].iloc[0]
    print(dt_utc)
    dt_et = pd.to_datetime(dt_utc).tz_convert('US/Eastern')
    print(dt_et)
    et = tz.gettz('US/Eastern')
    midnight_et = datetime.datetime.combine(dt_et.date(), datetime.datetime.min.time(), tzinfo = et) + timedelta(days = 1)
    midnight = midnight_et.astimezone(pytz.timezone('UTC'))
    print('midnight={}'.format(midnight))
    
    # Set up 3 o'clock check 
    # Convert est then back to utc for inits
    et3 = datetime.datetime.combine(dt_et.date(), datetime.time(3,0,20), tzinfo = et)
    utc3 = et3.astimezone(pytz.timezone('UTC'))
    print('three oclock={}'.format(utc3))
    
    # Process each row in df2
    # Df2 = timestamp and symbol cols
    # If symbol timestamp > 3 o'clock clear expired and insert 'C' and 'A' init record
    inserted_snapshot = False
    for index, row in df.iterrows():
        if not inserted_snapshot and row['timestamp'] > utc3:
            inserted_snapshot = True
            # Clear expired symbols
            clear_expired(utc3)
            
            # Insert snapshot -- clear and inits
            for slist in symbol_lists:
                # Insert a 'C' record
                listid.append('L')
                timestamp.append(utc3)
                date.append(utc3.date) 
                time.append(utc3.time) 
                guid.append(slist.guid)
                owner.append('-1')
                action.append('C')
                symbol.append('')
                description.append('')
                utc3 = utc3 + timedelta(milliseconds = 1)
                    
                for s in slist.content:
                    # Insert an 'A' record
                    listid.append('L')
                    timestamp.append(utc3)
                    date.append(utc3.date) 
                    time.append(utc3.time) 
                    guid.append(slist.guid)
                    owner.append('-1')
                    action.append('A')
                    symbol.append(s)
                    description.append('Init')
                utc3 = utc3 + timedelta(milliseconds = 1)
                
        # Process primary and secondary symbols
        for slist in [x for x in symbol_lists if x.category == 'primary']:
            process_row(row['symbol'], row, slist)

        for slist in [x for x in symbol_lists if x.category == 'secondary']:    
            for s in row['symbols']:
                if s[0].isalpha():
                    process_row(s, row, slist)

    # End of day check
    clear_expired(midnight)
    
    # Assign
    dfout = pd.DataFrame({'listid': listid, 'timestamp': timestamp, 'date': date, 
                                            'time': time,
                                            'guid': guid, 'owner': owner,
                                            'action': action, 'symbol': symbol,
                                            'description': description})            
    return dfout

In [1247]:
# Write to csv and formatting
def write_output(df, filename):
    # Sort by timestamp
    df = df.sort_values(by = ['timestamp','symbol'] , ascending = True)
    # Timezone convert
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.tz_convert('US/Eastern')
    # Create date time cols & correct format of date and time
    df['date'] = df['timestamp'].dt.date
    df['time'] = df['timestamp'].dt.time
    df['time'] = df['time'].apply(lambda x: x.strftime("%H:%M:%S.%f"))
    df['time'] = df['time'].astype(str).str[:-3]
    # Fix dtypes
    df['date'] = pd.to_datetime(df['date'], format = '%m/%d/%Y')
    df['date'] = df['date'].apply(lambda x: x.strftime("%m/%d/%Y"))
    # lowercase the guid and drop timestamp
    df['guid'] = df['guid'].apply(str.lower)
    df = df.drop('timestamp', axis = 1)
    # Write to csv
    df.to_csv(filename,index = False, sep = '|', header = False)
    return df

In [1242]:
# All records are added from these guids and expirations
# Calling all functions 
# Declare namedtuple
SymbolsList = namedtuple('SymbolsList',['content','expire_duration','listid','guid','category'])
# Add values
symbol_lists = (SymbolsList(dict(),timedelta(hours = 1),1179,'38BAC7D0-3405-4CC9-A2DF-926D5661429E','primary'),
                SymbolsList(dict(),timedelta(hours = 24),1180,'413BD802-B6E8-476F-8266-200B243F7243','primary'),
                SymbolsList(dict(),timedelta(hours = 12),1181,'9E7E633E-1005-4944-AA93-06EFA58D73E0','primary'),
                SymbolsList(dict(),timedelta(minutes = 15),1227,'067F84FB-C7F2-4570-A83F-B90AE0ADB23E','primary'),
                SymbolsList(dict(),timedelta(minutes = 15),1223,'CBFFA48E-A627-459B-8DC0-8CED39F68AF3','secondary'),
                SymbolsList(dict(),timedelta(hours = 1),1224,'CD76BAB7-5F8F-41F7-972B-7EF147E13A91','secondary'),
                SymbolsList(dict(),timedelta(hours = 12),1225,'5D7BF4A5-D016-480C-BD3C-C850470F139A','secondary'),
                SymbolsList(dict(),timedelta(hours = 24),1226,'952D33B9-D2E8-4A49-BE28-E2B42975BB03','secondary'))

# Iterate thru infiles
# Remove old output files

output_directory = "C:/Users/rhett/NewsTest"
output_files = os.listdir(output_directory)

for f in output_files:
    os.remove(os.path.join(output_directory, f))

# Get list of files in input directory
input_directory = "C:/Users/rhett/NewsLists/NewsListsOutput"
input_files = os.listdir(input_directory)

'''
for f in input_files:
    file_path = os.path.join(input_directory, f)
    df = pd.read_csv(file_path)
'''
# List 06-05 and 06-06
#csv_dates = ['2023-06-05','2023-06-06','2023-06-07','2023-06-08']
for d in csv_dates:
    #infile = 'C:/Users/rhett/Downloads/ListChangeEvents/SourceOutput/{}.csv'.format(d)
    #outfile = 'C:/Users/rhett/Downloads/ListChangeEvents/NewsTest/{}_news_test.csv'.format(d)
    df = load_news(input_files)
    dfout = process_news_day(df)
    #print('>>>>>>>>>> {} <<<<<<<<<<'.format(d))
    #print('process new day\n', dfout[['timestamp','action']])
    dfout = write_output(dfout, output_files)
    #print('write ouput\n', dfout[['date','time','action']])

2023-06-05 04:05:04.157000+00:00
2023-06-05 00:05:04.157000-04:00
midnight=2023-06-06 04:00:00+00:00
three oclock=2023-06-05 07:00:20+00:00
>>>>>>>>>> 2023-06-05 <<<<<<<<<<
process new day
                              timestamp action
0     2023-06-05 04:05:04.157000+00:00      A
1     2023-06-05 04:05:04.157000+00:00      A
2     2023-06-05 04:05:04.157000+00:00      A
3     2023-06-05 04:05:04.157000+00:00      A
4     2023-06-05 04:05:04.157000+00:00      A
...                                ...    ...
55156 2023-06-06 03:55:53.363000+00:00      R
55157 2023-06-06 03:56:26.897000+00:00      R
55158 2023-06-06 03:57:02.313000+00:00      R
55159 2023-06-06 03:58:21.307000+00:00      R
55160 2023-06-06 03:59:39.993000+00:00      R

[55161 rows x 2 columns]
write ouput
              date          time action
4      06/05/2023  00:05:04.157      A
5      06/05/2023  00:05:04.157      A
6      06/05/2023  00:05:04.157      A
7      06/05/2023  00:05:04.157      A
64     06/05/2023  00:05

In [1243]:
# Read file
file_dates = ['2023-06-05', '2023-06-06','2023-06-07','2023-06-08']

for date in file_dates:
    files = 'C:/Users/rhett/Downloads/ListChangeEvents/NewsTest/{}_news_test.csv'.format(date)
    #outf = 'C:/Users/rhett/Downloads/ListChangeEvents/{}_news_test2.csv'.format(date)
    file_cont = ''
    with open(files, 'r') as input_file:
        file_cont = input_file.read()

    # search and replace |C||
        file_cont = file_cont.replace('|C||', '|C')

    # write it back out
    with open(files, 'w') as output_file:
        output_file.write(file_cont)